In [ ]:
class NeuralNet:
    """
    NN for binary classification
    Attributes:
    ...
    """
    
    def __init__(self, layers_d, normalize = True, learning_rate = 0.01, num_iter = 30000, epsilon = (10)^(-10), k = 500):
        self.layers_d = layers_d # тут лише приховані шари а 0-го та останнього (з одним нейроном) немає
        self.L = len(self.layers_d) + 1 # кількість шарів нейронів в мережі без урахування вихідного
        self.learning_rate = learning_rate
        self.num_iter = num_iter
        self.normalize = normalize
        self.epsilon = epsilon
        self.k = k
    
    def __normalize(self, X, mean = None, std = None):
        """
        Зверніть увагу, що нормалізація вхідних даних є дуже важливою для швидкодії нейронних мереж.
        """
        '''
        X.shape =  (n, m)
        '''
        n = X.shape[0]
        m = mean
        if m is None:
            m = np.mean(X, axis=1).reshape((n, 1))
            '''
            m.shape =  (n, 1)
            '''
        s = std
        if s is None:
            s = np.std(X, axis=1).reshape((n, 1))
            '''
            s.shape =  (n, 1)
            '''
        X_new = (X - m) / s
        return X_new, m, s

    def __sigmoid(self, Z):
        """
        В наступних практичних потрібно буде додати підтримку й інших активаційних функцій - це один з гіперпараметрів. 
        Їх можна вибирати для всіх шарів одночасно або мати різні активаційні функції на кожному з них.
        """
        return 1 / (1 + np.exp(-Z))
    
    def __initialize_parameters(self):
        
        self.parameters = {} # стоврюємо словник зі значеннями W_i та b_i,ключами в якому будуть назви W_1, w_2, ... та b_1, b_2 і т.д
        
        for i in range(1, self.L + 1):
            self.parameters['W_' + str(i)] = np.random.randn(self.layers_d[i], self.layers_d[i - 1])* np.sqrt(2/self.layers_d[i - 1])
            '''
            W_i.shape  = (n_l, n_l-1) # (кількість нейронів на поточному шарі, кількість на попередньому)
            '''
            self.parameters['b_' + str(i)] = np.zeros((self.layers_d[i],1))
            '''
            b_i.shape  = (n_l,1) # (кількість нейронів на поточному шарі, 1)
            '''
       
    def __forward_propagation(self, X):
        
        cache = {} # стоврюємо словник зі значеннями Z_i та A_i,ключами в якому будуть назви A_0, A_1, A_2, ... та Z_1, Z_2 і т.д
        cache['A_0'] = X
        
        for i in range(1, self.L + 1):
            cache['Z_' + str(i)] = np.dot(self.parameters['W_' + str(i)], cache['A_' + str(i - 1)]) + self.parameters['b_' + str(i)]
            '''
            Z_i.shape  = (n_l, 1) = (n_l, n_l-1) * (n_l-1, 1) + (n_l,1)
            '''
            cache['A_' + str(i)] = self.__sigmoid(cache['Z_' + str(i)])
            '''
            A_i.shape  = (n_l, 1) = (n_l, 1)
            '''       
        
        return cache['A_' + str(self.L)], cache
    
    def compute_cost(self, A, Y):
        m = Y.shape[1]
        res = Y * np.log(A) + (1 - Y) * np.log(1 - A)
        J = -(1 / m) * np.sum(res)
        '''
        J.shape  = sum((1, m) x (1, m) - (1, m) x (1, m)) = sum((1, m)) = (1, 1)
        '''
        return J
        
    def __backward_propagation(self, X, Y, cache):
        
        m = X.shape[1]
        gradients = {}
        
        gradients['dZ_' + str(self.L)] = cache['A_' + str(self.L)] - Y
        '''
        dZ_L.shape  = (1, m) - (1, m) = (1, m)
        '''
        gradients['dW_' + str(self.L)] = (1/m) * np.dot (gradients['dZ_' + str(self.L)], cache['A_' + str(self.L - 1)].T)
        '''
        dW_L.shape  = (1, m) * ((n_l-1, m).T) = (1, m) * (m, n_l-1) = (1, n_l-1)
        '''
        gradients['db_' + str(self.L)] = (1/m) * np.sum(gradients['dZ_' + str(self.L)], axis = 1, keepdims = True)
        '''
        db_L.shape  = sum((1, m)) = (1, 1)
        '''
        
        for i in range(self.L - 1, 0, -1):
            dA_i = np.dot (self.parameters['W_' + str(i + 1)].T, gradients['dZ_' + str(i + 1)])
            '''
            dA_i.shape  = (n_l-1, n_l)*(n_l, m) = (n_l-1, m)
            '''
            gradients['dZ_' + str(i)] = np.multiply(dA_i, cache['A_' + str(i)] * (1 - cache['A_' + str(i)]))
            '''
            dZ_i.shape  = (n_l, m)x(n_l, m) = (n_l, m)
            '''
            gradients['dW_' + str(i)] = (1/m) * np.dot (gradients['dZ_' + str(i)], cache['A_' + str(i - 1)].T)
            '''
            dW_i.shape  = (n_l, m)*((n_l-1, n).T) = (n_l, m)*(m, n_l-1) = (n_l, n_l-1)
            '''
            gradients['db_' + str(i)] = (1/m) * np.sum(gradients['dZ_' + str(i)], axis = 1, keepdims = True)
            '''
            db_i.shape = sum((n_l, m)) = (n_l, 1)
            '''       
        
        return gradients
    
    def __update_parameters(self, gradients):
        
        for i in range(1, self.L + 1):
            self.parameters['W_' + str(i)] -= self.learning_rate * gradients['dW_' + str(i)]
            '''
            W_i.shape  = (n_l, n_l-1) # (кількість нейронів на поточному шарі, кількість на попередньому)
            '''
            self.parameters['b_' + str(i)] -= self.learning_rate * gradients['db_' + str(i)]
            '''
            b_i.shape  = (n_l,1) # (кількість нейронів на поточному шарі, 1)
            '''
        
    def fit(self, X_vert, Y_vert, print_cost = True):
        
        X, Y = X_vert.T, Y_vert.T
        
        n_x = X.shape[0] # визначаємо кількість нейронів у вихідному шарі
        final_classes = Y.shape[0] # визначаємо кількість нейронів у вихідному шарі
        
        self.layers_d.insert(0, n_x)
        self.layers_d.append(final_classes) 
        '''
        додаємо вхідний та вихідний шари до прихованих 
        і отримуємо клькість всіх шарів нейронної мережі і кількість нейронів на кожному шарі
        '''
        
        if self.normalize:
            X, self.__mean, self.__std = self.__normalize(X)
        
        costs = []
        
        m = X.shape[1]
        n_x = X.shape[0]
        
        self.__initialize_parameters()
        
        previous_cost = 0;

        for i in range(self.num_iter):
            
            A, cache = self.__forward_propagation(X)

            cost = self.compute_cost(A, Y)

            gradients = self.__backward_propagation(X, Y, cache)

            self.__update_parameters(gradients)

            if print_cost and i % 1000 == 0:
                print("{}-th iteration: {}".format(i, cost))

            if i % 1000 == 0:
                costs.append(cost)
            if (abs(previous_cost - cost) < self.epsilon):
                k = k - 1
                if (k == 0):
                    break;

        if print_cost:
            plt.plot(costs)
            plt.ylabel("Cost")
            plt.xlabel("Iteration, *1000")
            plt.show()
    
    def predict_proba(self, X_vert):
        X = X_vert.T
        if self.normalize:
            X, _, _ = self.__normalize(X, self.__mean, self.__std)    
        
        probs = self.__forward_propagation(X)[0]
        return probs.T
    
    def predict(self, X_vert):
        positive_probs = self.predict_proba(X_vert)[:, 0]
        return (positive_probs >= 0.5).astype(int)